In [2]:
!pip install transformers
!pip install sentencepiece
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 111.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 40.2 MB/s eta 0:00:00


In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE= 1e-05

targets_settings =[
   ['Spiritual', 'Physical', 'Intellectual', 'Social', 'Vocational','Emotional'],
   ['Physical', 'Intellectual', 'Social', 'Vocational','Spiritual_Emotional'],
   [ 'Intellectual', 'Social', 'Vocational','Physical_Spiritual_Emotional'],
   [ 'Social', 'Intellectual_Vocational','Physical_Spiritual_Emotional']
]
classifier_index = 4 #Models
target_index = 0 # [0: 6-dim, 1:5-dim, 2:4-dim, 3:3-dim]
ran_index = 1 #d[0:200, 1:345, 2:546]

Classifiers = ["nghuyong/ernie-2.0-en", "bert-base-uncased","roberta-base" ,"emilyalsentzer/Bio_ClinicalBERT", "xlnet-base-cased",'nlptown/bert-base-multilingual-uncased-sentiment', "mental/mental-bert-base-uncased"]
Classifiers_Abs = ["ERNIE", "BERT", "RoBERTa", "ClinicalBERT", "XLNET", "PsychBERT", "Mental-BERT"]
TheClassifier = Classifiers[classifier_index]
TheClassifier_Abstract = Classifiers_Abs[classifier_index]

rand_states = [200, 345, 546]


rand_state = rand_states[ran_index]

target_List = targets_settings[target_index]
dimension = len(target_List)

import pandas as pd
import numpy as np

data=pd.read_csv("/content/drive/MyDrive/Wellness_Dataset_Paper/MultiLabel_WD - Sheet1 (1) (1).csv") # MultiLabel_WD.csv is the first Dataset

data=data.astype({'Spiritual':'float', 'Physical':'float', 'Intellectual':'float', 'Social':'float', 'Vocational':'float',
       'Emotional':'float'})

tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
# print(data)
labels_name = data.columns

labels_name = labels_name[1:]
labels = data[labels_name]

counts = np.zeros(dimension)
for i in range(len(data)):
  for j in range(dimension):
    if data.loc[i][labels_name[j]]>0:
      counts[j] += 1

labels_dic ={labels_name[i]:counts[i] for i in range(dimension)}
data_d5 = data.copy(deep=True)
data_d4 = data.copy(deep=True)
data_d3 = data.copy(deep=True)

data_d5['Spiritual_Emotional'] = data_d5[[ 'Spiritual', 'Emotional']].max(axis=1)
data_d5 = data_d5.drop(['Spiritual', 'Emotional'], axis=1)

data_d4['Physical_Spiritual_Emotional'] = data_d4[['Physical', 'Spiritual', 'Emotional']].max(axis=1)
data_d4 = data_d4.drop(['Physical','Spiritual', 'Emotional'], axis=1)

data_d3 = data_d4.copy(deep=True)
data_d3['Intellectual_Vocational'] = data_d3[['Intellectual', 'Vocational']].max(axis=1)
data_d3 = data_d3.drop(['Intellectual', 'Vocational'], axis=1)

if dimension==5:
   data=data_d5
elif dimension==4:
   data=data_d4
elif dimension==3:
   data=data_d3


from transformers import AutoTokenizer, AutoModel, XLNetTokenizer,XLNetModel
#tokenizer = AutoTokenizer.from_pretrained(TheClassifier)
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['text']
        self.targets = self.df[target_List].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

test_size = 0.2
val_df = data.sample(frac=test_size, random_state=rand_state).reset_index (drop=True)
train_df = data.drop (val_df.index).reset_index (drop=True)
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)
if classifier_index==4:
  class XLNETClass(torch.nn.Module):
    def __init__(self):
        super(XLNETClass, self).__init__()
        self.xlnet_model = XLNetModel.from_pretrained("xlnet-base-cased", output_hidden_states=True, output_attentions=True, return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, dimension + 1)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output_dict = self.xlnet_model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids,
            output_hidden_states=True,
            return_dict=True
        )
        last_hidden_state = output_dict.last_hidden_state
        attention_weights = output_dict.attentions
        output_dropout = self.dropout(last_hidden_state[:, -1, :])
        output = self.linear(output_dropout)
        return output, attention_weights[-1]

  model = XLNETClass()
  model.to(device)

def loss_fn(m_outputs, targets):
        reward = dimension

        tensor_temp = torch.zeros(32,dtype=torch.float)
        tensor_temp.to(device)
        outputs = torch.nn.functional.softmax(m_outputs, dim=1,dtype=torch.float)

        outputs, reservation = outputs[:, :-1], outputs[:, -1]

        # gain = torch.gather(outputs, dim=1, index=targets).squeeze()
        # print("targets:",targets)
        # print("outputs:", outputs)
        # raise KeyboardInterrupt
        # return targets, outputs
        gain = torch.einsum("ij, ij -> i", targets.to(torch.float), outputs)

        # doubling_rate = (gain.max() + reservation / reward).log()
        doubling_rate = -torch.log(gain + reservation/reward)
        return  doubling_rate.mean(), reservation

def loss_fn2(outputs, targets):
    # return torch.nn.BCEWithLogitsLoss()(outputs[:,:-1], targets)
    return torch.nn.CrossEntropyLoss()(outputs[:, :-1], targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
val_targets=[]
val_outputs=[]


def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss, _ = loss_fn(outputs, targets.type(torch.int64))
        # tar, outp = loss_fn(outputs, targets.type(torch.int64))
        # return tar, outp

        # print(len(targets))
        loss2 = loss_fn2(outputs, targets)



        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss, _ = loss_fn(outputs, targets.type(torch.int64))

            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAverage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
        # save checkpoint
    #   save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        # save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

import shutil, sys

model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

def finalLabels2(predicted_list,val_list):

  indices=np.array(predicted_list).argsort()[::-1][:int(sum(val_list))]
  # argsort()[:-1][:n]
  # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
  for j in range(len(predicted_list)):
    if j in indices:
      predicted_list[j]=1.0
    else:
      predicted_list[j]=0.0
  return predicted_list

final_list=[]
# for i in val_df['text']:
#   example = i
for i in range(len(val_df)):
  example  = val_df.loc[i]['text']
  target  = [val_df.loc[i][j+1] for j in range(dimension)]
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output = model(input_ids, attention_mask, token_type_ids)
      temp = torch.Tensor(target).type(torch.int64).to(device)
      loss, reservation = loss_fn(output,temp.reshape([1,dimension]))
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      # print(final_output[0][:-1], target)
      temp = finalLabels2(final_output[0][:-1],target)
    #   print(temp)
    #   print(len(temp))
    #   print(target)
    #   print(len(target))

      final_list.append(temp+target+torch.Tensor.tolist(reservation))

from operator import itemgetter
sorted_final_list = sorted(final_list, key=itemgetter(2*dimension - 1), reverse = False)

from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix

def get_accuracies(true_labels, predictions):
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.multilabel_confusion_matrix.html
    cm = multilabel_confusion_matrix(true_labels, predictions)
    total_count = np.array(true_labels).shape[0]
    accuracies = []
    # print(np.array(true_labels).shape[1])
    # raise KeyboardInterrupt
    for i in range(np.array(true_labels).shape[1]):
        true_positive_count = np.sum(cm[i,1,1]).item()
        true_negative_count = np.sum(cm[i,0,0]).item()
        accuracy = (true_positive_count + true_negative_count) / total_count
        accuracies.append(accuracy)
    return accuracies


from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
label_names = target_List
for threshold in [1, .95, 0.9, 0.85, 0.8,.75]:
  list_data = sorted_final_list[:round(threshold*len(sorted_final_list)-1)]
  val_list = [list_data[i][dimension:2*dimension] for i in range(len(list_data))]
  prediction = [list_data[i][0:dimension] for i in range(len(list_data))]
  # print(val_list)
  # print(prediction)
  # print(dimension,len(val_list), len(prediction))
  # raise KeyboardInterrupt
  print('############# '+TheClassifier_Abstract+'_Dim'+str(dimension)+'_run'+str(rand_state)+'_threshold'+str(threshold)+'   #############')

  print(classification_report(val_list, prediction,target_names=label_names))

  accuracies = get_accuracies(val_list,prediction)
  accuracies = [round(accuracies[i],2) for i in range(dimension)]
  print("accuracies for each class:",accuracies)

  val = pd.DataFrame(val_list, columns = target_List)
  fin = pd.DataFrame(prediction, columns = target_List)

  print('MCC:')

  for i in range(dimension):
     label = target_List[i]
     print(label, matthews_corrcoef(val[label],fin[label]))
#   print("Physical", matthews_corrcoef(val["Physical"],fin["Physical"]))
#   print("Spiritual", matthews_corrcoef(val["Spiritual"],fin["Spiritual"]))
#   print("Intellectual", matthews_corrcoef(val["Intellectual"],fin["Intellectual"]))
#   print("Social", matthews_corrcoef(val["Social"],fin["Social"]))
#   print("Vocational", matthews_corrcoef(val["Vocational"],fin["Vocational"]))
#   print("Emotional", matthews_corrcoef(val["Emotional"],fin["Emotional"]))

print('Done!!!!!!!!!!!!')

cpu


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


############# Epoch 1: Training Start   #############
